In [1]:
import ee
import datetime
import pandas as pd

ee.Authenticate()
ee.Initialize(project='ee-prathama76')

# EXTRACT SAR FOR A SINGLE FILE

In [2]:

# Load CSV
input_csv_path = 'Agrifield/Data/Agrifield.csv'
df = pd.read_csv(input_csv_path)

# Convert CSV rows to ee.FeatureCollection
def row_to_feature(row):
    geom = ee.Geometry.Point([row['longitude'], row['latitude']])
    return ee.Feature(geom, {
        'Latitude': row['latitude'],
        'Longitude': row['longitude'],
        'Crop_Name': row['Crop_Name']
    })

features = [row_to_feature(row) for _, row in df.iterrows()]
feature_collection = ee.FeatureCollection(features)

# Dates
start_date = ee.Date('2021-04-01')
end_date = ee.Date('2022-03-31')

# Load Sentinel-1 data (VV, VH) and filter globally
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.eq("instrumentMode", "IW")) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH")) \
    .filterBounds(feature_collection) \
    .select(["VV", "VH"])

# Function to reduce each image
def process_image(image):
    date = image.date().format("YYYY-MM-dd")
    pass_dir = image.get("orbitProperties_pass")
    reduced = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=10
    ).map(lambda f: f.set("date", date).set("Pass_Direction", pass_dir))
    return reduced

# Apply processing function
sar_data = sentinel1.map(process_image).flatten()

# Export to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=sar_data,
    description='Begalavi_SAR_Data',
    folder='Agrifield',
    fileNamePrefix='Agrifield',
    fileFormat='CSV'
)
task.start()

print("✅ SAR Export started. Monitor at https://code.earthengine.google.com/tasks")


✅ SAR Export started. Monitor at https://code.earthengine.google.com/tasks


# EXTRACT SAR FOR A DIRECTORY

In [3]:
import ee
import os
import pandas as pd

ee.Initialize()

# Set directory path containing the CSV files
input_directory = 'Karnataka_Datasets/Across/Kharif/Cropland_Masked/Cropland_Mask_2018_19/'

# Define processing dates
start_date = ee.Date('2018-04-01')
end_date = ee.Date('2019-03-31')

# Load Sentinel-1 ImageCollection
def get_sentinel1_collection(bounds):
    return ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq("instrumentMode", "IW")) \
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
        .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH")) \
        .filterBounds(bounds) \
        .select(["VV", "VH"])

# Convert each row of a DataFrame to an ee.Feature
def row_to_feature(row):
    geom = ee.Geometry.Point([row['longitude'], row['latitude']])
    return ee.Feature(geom, {
        'Latitude': row['latitude'],
        'Longitude': row['longitude'],
        'Crop_Name': row['Crop_Name']
    })

# Reduce image per feature
def process_image(image, feature_collection):
    date = image.date().format("YYYY-MM-dd")
    pass_dir = image.get("orbitProperties_pass")
    reduced = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=10
    ).map(lambda f: f.set("date", date).set("Pass_Direction", pass_dir))
    return reduced

# Process each CSV file
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        csv_path = os.path.join(input_directory, filename)
        print(f"🚀 Processing file: {csv_path}")

        # Read CSV
        df = pd.read_csv(csv_path)

        # Convert to FeatureCollection
        features = [row_to_feature(row) for _, row in df.iterrows()]
        feature_collection = ee.FeatureCollection(features)

        # Get Sentinel-1 images for this region
        sentinel1 = get_sentinel1_collection(feature_collection)

        # Apply image processing
        sar_data = sentinel1.map(lambda image: process_image(image, feature_collection)).flatten()

        # Set output task parameters
        base_name = os.path.splitext(filename)[0]
        task = ee.batch.Export.table.toDrive(
            collection=sar_data,
            description=f'{base_name}_SAR_Export',
            folder='Cropland_Masked_2018_19',
            fileNamePrefix=f'{base_name}_SAR_Data_Masked',
            fileFormat='CSV'
        )
        task.start()
        print(f"✅ Export started for {filename}. Monitor at https://code.earthengine.google.com/tasks")


🚀 Processing file: Karnataka_Datasets/Across/Kharif/Cropland_Masked/Cropland_Mask_2018_19/interpolated_Bidar_3_SAR_Data_part1_Masked.csv


KeyError: 'longitude'